## 数值稳定性、模型初始化和激活函数

### 1.梯度消失和梯度爆炸

由于我们需要进行梯度的反向传播，需要连乘；对比较深的神经网络，当梯度值过大或过小时，连乘的结果就会上溢或下溢。
- 梯度爆炸的问题：
  - 值超出值域（infinity）
    - 对于16位浮点数尤为严重(6e-5, 6e4)
  - 对学习率敏感
    - 学习率太大=》大参数值=》更大的梯度
    - 学习率太小=》训练没有进展
    - 我们可能需要在训练过程中，不断调整学习率
- 梯度消失的问题：
  - 梯度值变成0，特别是16位浮点数
  - 训练没有进展，不管如何选择学习率
  - 对底层尤为严重
    - 仅仅顶部层训练的较好，底部由于梯度值太小，没啥效果
    - 无法让神经网络更深
- 参数化自带的对称性问题：不可初始化权重W为常数值！

### 2.让训练更加稳定

目标：让梯度值在合理的范围内，如[1e-6, 1e3]
- 将部分乘法变加法：ResNet、LSTM
- 归一化：梯度归一化、梯度剪裁
- 合理的权重初始化和激活函数

### 3.参数初始化

解决（或至少减轻）上述问题的一种方法是进行参数初始化；优化期间的注意和适当的正则化也可以进一步提高稳定性。

- 在训练开始的时候更容易有数值不稳定
  - 远离最优解的地方，损失函数表面可能很复杂
- 前面我们常用正态分布来做初始化，对小网络没问题，但不能保证深度神经网络也。

#### 3.1 Xavier初始化

- 思想来源：
  - 考察单个线性层，输入和输出维度分别为$n_{in},n_{out}$，输入为x，输出表示为：$o_i = \sum_{j=1}{n_{in}}w_{ij}x_j$
    - 假设权重$w_{ij}$独立同分布，均值为0方差为$\sigma^2$；输入值$x_j$独立同分布，均值为0方差为$\gamma^2$，且与权重$w_{ij}$互相独立
    - 计算输出$o_i$的均值和方差，可得：$E[o_i]=0, ~ Var[o_i] = n_{in}\sigma^2\gamma^2$
  - 想要保持本层的输入方差和输出方差一致，则需要设置$n_{in}\sigma^2=1$；
  - 相应地，考察上面模型的梯度反向传播，想保持梯度的方差前后一致，则需要设置$n_{out}\sigma^2=1$；
  - 折中，取$\frac{1}{2}(n_{in}+n_{out})\sigma^2=0$
- 实践中：虽然不总是线性的，但实践证明有效
  - 一般从高斯分布中采样，$\mathcal{N}(0, \frac{2}{n_{in}+n_{out}})$
  - 也可以利用Xavier的直觉来选择从均匀分布中抽取权重时的方差：$\mathcal{U}(-\sqrt{\frac{6}{n_{in}+n_{out}}}, \sqrt{\frac{6}{n_{in}+n_{out}}})$
    - 分布$\mathcal{U}[-a,a]$的方差是$\frac{a^2}{3}$

#### 3.2拓展

深度学习框架通常实现十几种不同的启发式方法。此外，参数初始化一直是深度学习基础研究的热点领域。

其中包括专门用于参数绑定（共享）、超分辨率、序列模型和其他情况的启发式算法。

### 4.激活函数

- 采用类似的思想观察激活函数，目标是使得输入和输出的均值、方差保持一致。
- 考察线性函数的均值和方差，正向传播和反向传播：
  - 对$y = ax+b$，若要保持一致，需要$b=0 \& a=1$
- 使用泰勒展开，观察常用激活函数：
  $$
  \begin{aligned}
    sigmoid(x) &= \frac{1}{2} + \frac{x}{4} - \frac{x^3}{48} + O(x^5) \\
    tanh(x) &= 0 + x - \frac{x^3}{3} +O(x^5) \\
    relu(x) &= 0 + x
  \end{aligned}
  $$
- 调整sigmoid：$4 * sigmoid(x) -2$

### 4.小结

- 梯度消失和梯度爆炸是深度网络中常见的问题。在参数初始化时需要非常小心，以确保梯度和参数可以得到很好的控制。
- 需要用启发式的初始化方法来确保初始梯度既不太大也不太小。